# CPU Version

In [11]:
import numpy as np
import matplotlib.pyplot as plt
import time as time

# Params
K = 100 # A strike for the option to price
T = 1 # Maturity of the option
S0 = 100 # An initial value for the  asset
V0 = 0.1 # An initial value for the variance process
r = 0.1 # Interest rate
theta = 0.1 # Mean of Variance
k = 0.2 # Mean reversion speed (CIR)
rho = 0.1 # Correlation of two brownians
sigma = 0.3 # Volatilty of volatility
n = 100 # Number of timesteps(I dont have to use it for the pricing since it is not path dependent here)
M =1000 # number of scenarios

In [12]:
dt = T/n


In [13]:
def calculate_price( M, n, dt, rho, k, theta, sigma, S0, V0, K, r, T):
    payoffs = np.zeros(M)
    for i in range(M):
        W1 = np.random.normal(0, np.sqrt(dt), n)
        W2 = rho*W1 + np.sqrt(1-rho**2)*np.random.normal(0, np.sqrt(dt), n)
        S_t = np.zeros(n)
        V_t = np.zeros(n)
        S_t[0] = S0
        V_t[0] = V0

        for j in range(1, n):
            S_t[j] = S_t[j-1] + r*S_t[j-1]*dt + np.sqrt(np.maximum(0,V_t[j-1]))*S_t[j-1]*W1[j]
            V_t[j] = V_t[j-1] + k*(theta-V_t[j-1])*dt + sigma*np.sqrt(np.maximum(0,V_t[j-1]))*W2[j]

        payoffs[i] = max(S_t[-1]-K, 0)

    price = np.mean(payoffs*np.exp(-r*T))
    std = np.std(payoffs*np.exp(-r*T))
    return price , std

start = time.time()

price , std = calculate_price(M, n, dt, rho, k, theta, sigma, S0, V0, K, r, T)

end = time.time()
print(f"The price of the call is {price}")
print(f"The std of the call is {std}")

print("the computation took (seconds)" , end - start)


The price of the call is 17.32318070803902
The std of the call is 27.083402048638185
the computation took (seconds) 0.9031007289886475


# GPU Version

In [14]:
!nvcc MC.cu -o MC

In [15]:
import subprocess

# Params change the before running !!
S_0 = 100
K = 100
r = 0.1
kappa = 0.2
theta = 0.1
sigma_v = 0.3
rho = 0.1
T = 1
N = 100
M = 1000000

# Run the CUDA script and capture the output
result = subprocess.run(['./MC', str(S_0), str(K), str(r), str(kappa), str(theta), str(sigma_v), str(rho), str(T), str(N),str(M)], capture_output=True, text=True)

# Extract the output containing random numbers
output = result.stdout.split('\n')
print(output)


['The estimated price is equal to 16.958187', 'error associated to a confidence interval of 95% = 0.051836', 'Execution time 4.194752 ms', '']
